In [ ]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.io import wavfile

In [ ]:
from keras.layers import Conv1D, MaxPool1D, Activation, Dense, Input, Flatten
import tensorflow as tf

# Prepare Training set

In [ ]:
annotations_path = '../data/MagnaTagATune/annotation_reduced.csv'
annotations = pd.read_csv(annotations_path, sep='\t')
train_set, test_set = train_test_split(annotations['mp3_path'], train_size=0.1, test_size=0.001) 

In [ ]:
x_train = []
x_test = []
for value in tqdm(train_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_train.append(data)
x_train = np.array(x_train)[:,:,np.newaxis]

for value in tqdm(test_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_test.append(data)
x_test = np.array(x_test)[:,:,np.newaxis]

print("X train shape:",x_train.shape)
print("X test shape:",x_test.shape)

In [ ]:
y_train = annotations.loc[annotations['mp3_path'].isin(train_set)].drop(columns=['mp3_path','Unnamed: 0']).values
y_test = annotations.loc[annotations['mp3_path'].isin(test_set)].drop(columns=['mp3_path','Unnamed: 0']).values
print("Y train shape:",y_train.shape)
print("Y test shape:",y_test.shape)

In [ ]:
config = tf.ConfigProto(device_count={'GPU':0})
#config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)

model = keras.Sequential()
model.add(Conv1D(filters=10, kernel_size=100, strides=80, input_shape=(x_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(filters=4, kernel_size=3, strides=1))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
accuracy_score(y_test,predictions,normalize=False)